# Aprendizado de Máquina com Spark

Spark oferece um ambiente completo para aprendizado de máquina em sua biblioteca `MLlib`, que implementa diversas tarefas de modo distribuído e escalável. Neste notebook veremos alguns exemplos de suas funcionalidades, que podem ser divididas em três grandes categorias:
- Transformações de Características
- Algoritmos
- Otimização 

Vamos começar importando as bibliotecas do Spark e inicializando uma `SparkSession`.

In [ ]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

print(spark)

## Transformações de Características

Spark oferece uma grande quantidade de transformações de características que podem ser aplicadas em Dataframes. Essas transformações vão além das funcionalidades vistas antes na biblioteca SQL. 


As transformações de características estão localizadas no módulo `pyspark.ml.feature`. Podemos checar todas as funcionalidades na [documentação](https://spark.apache.org/docs/latest/ml-features.html). No caso deste notebook, veremos as cinco mais comuns:
- Indexador de String
- Transformador de representações OneHot
- Transformador de valores em _buckets_
- TF-IDF
- Criador de vetores


Mas antes de vermos eles, vamos importar e preparar as bases de dados `flights.csv` e `sms.csv`, usando SQL.

Preparando `flights.csv`:

In [ ]:
# importando e olhando o Schema

# definindo a coluna alvo

# normalizando valores por min-max

# remover colunas desnecessarias

#remover linhas com valores nulos


Preparando `sms.csv`:

In [ ]:
# importando e olhando o Schema

# renomear colunas


### StringIndexer

O Indexador de String transforma o conteúdo de cada célula de uma coluna de Strings em um valor categórico. Essa decisão é feita com base na frequência do elemento. 

Vamos modificar algumas colunas de `flights_df`.

In [ ]:
print('Checando se "carrier" é categórico')

print('Indexando "carrier"')

print('Indexando "origin" e "dest"')


### OneHotEncoder

Dados categóricos não podem ser manipulados pela maioria de algoritmos de aprendizado de máquina, pelo simples fato de que eles não possuem relação matemática alguma entre si. Para podermos usar dados categóricos na maioria dos algoritmos, precisamos transformá-los em uma representação **one-hot**. 

Spark realiza essa conversão através do `OneHotEncoder`, porém o que faz de fato é gerar uma representação **dummy**: 

![Representação one-hot. Retirado de: https://www.kaggle.com/getting-started/187540](https://www.googleapis.com/download/storage/v1/b/kaggle-forum-message-attachments/o/inbox%2F5315434%2Fa9886ea90db74aad0b2f86d2686c337b%2Fohe-vs-dummy.png?generation=1601465979026694&alt=media)

Vamos converter os índices que criamos em representações one-hot.

In [ ]:
print('Criando one-hots para "carrier_idx"')

print('Criando one-hots para "origin" e "dest"')


### Bucketizer

Às vezes é interessante transformar valores contínuos em discretos para uma generalização melhor do modelo. O modelo assim passa a diferenciar por _categorias_ de valores e não tentar entender um comportamento a partir da variação em um valor. Ou ainda, é interessante diminuir o número de categorias presentes em uma variável já discreta.

Essa tarefa é conhecida como _bucketing_ ou _binning_. Vamos fazer essa transformação com a coluna `'hour'`, e produzir vetores one-hot a partir dela.

In [ ]:
print('Ver valores únicos de "hour"')

print('Reduzir para buckets de 3 horas')


### VectorAssembler

Spark requer que toda a informação que será passada para um algoritmo de ML seja convertida em um único vetor. Para fazer isso, usaremos `VectorAssembler`.

### TF-IDF ...de novo

Spark oferece funções para a criação de vetores TF-IDF. O processo é quebrado em três transformações: `Tokenizer`, `HashingTF`, `IDF`. Vamos transformar a coluna `'text'` em TF-IDF em nosso `sms_df`.

In [ ]:
print('Limpando o texto - remover pontuação, números e espaços adicionais')
#colinha: regex para remover específicas pontuações e números [_():;,.!?\\-0-9]

print('Tokenizando e removendo stop words')

print('Calculando TF')

print('Calculando TF-IDF')


## Algoritmos de aprendizado de máquina em Spark

Veremos quatro categorias de algoritmos de ML a seguir:
- Classificação
- Regressão
- Agrupamento
- Recomendação

### Classificação

Os algoritmos de classificação aprendem um modelo que é capaz de discernir instâncias entre múltiplas classes. Os algoritmos dessa categoria estão localizados no módulo `pyspark.ml.classification`. Podemos olhar a [documentação](https://spark.apache.org/docs/latest/ml-classification-regression.html#classification) para verificar quais algoritmos estão disponíveis. No caso desse notebook, olharemos para dois algoritmos clássicos:

- Árvore de Decisão
- Regressão Logística

Para avaliar a performance dos algoritmos, podemos utilizar dois objetos:`MulticlassClassificationEvaluator` e `BinaryClassificationEvaluator`. O primeiro lida com avaliação de modelos capazes discretizar entre múltiplas classes e contém métricas como precisão, revocação, medida-F por classe e/ou ponderada. Já o segundo, foca em análise de classificação binária e possui implementações específicas para esse caso, como AUC. 


#### Árvore de Decisão

Árvore de decisão é um dos algoritmos mais clássicos de aprendizado de máquina. O algoritmo escolhe a característica mais importante para particionar o espaço, o divide e recursivamente o invoca para resolver os subconjuntos resultantes.

![Arvore de decisão para jogar tênis, retirado de: https://www.researchgate.net/figure/Decision-tree-for-conditions-to-play-tennis_fig1_283569105](https://www.researchgate.net/profile/Peter-Wagacha/publication/283569105/figure/fig1/AS:293585607639040@1447007671951/Decision-tree-for-conditions-to-play-tennis.png)


Vamos rodar `DecisionTreeClassifier` para aprender um modelo com `flights_vec` e com `sms_tfidf`.

In [ ]:
#dividir em treino/teste

print('Rodando o modelo e vendo o resultado')

print('Matriz de confusão')


Vamos avaliar o modelo? Vamos ver sua eficácia em termos de precisão e revocação ponderadas, e AUC.

In [ ]:
# rodar arvore de decisão com sms_tfidf


#### Regressão Logística

Na regressão logística, utiliza-se uma função sigmoide para realizar classificação binária:  

![Disponível em: https://www.javatpoint.com/logistic-regression-in-machine-learning](https://static.javatpoint.com/tutorial/machine-learning/images/logistic-regression-in-machine-learning.png)

Vamos rodar `flights_vec` e `sms_tfidf` também nesse modelo e comparar os resultados.

In [ ]:
# rodar regressão logística com flights_vec


In [ ]:
# rodar regressão logística com sms_tfidf


### Regressão

Em regressão o modelo é treinado para prever um valor contínuo ao invés de uma classe. Os algoritmos dessa categoria estão localizados no módulo `pyspark.ml.regression`. Podemos olhar a [documentação](https://spark.apache.org/docs/latest/ml-classification-regression.html#regression) para verificar quais algoritmos estão disponíveis. No caso desse notebook, olharemos para os seguintes algoritmos:

- Regressão Linear
- Random Forest para regressão

Para avaliar a performance dos algoritmos, podemos utilizar `RegressionEvaluator`. Nele há a implementação de métricas de erro, como MSE, RMSE, R2, MAE.


#### Regressão Linear

Regressão linear é o algoritmo mais simples de aprendizado de máquina. Ele tenta encontrar um modelo que descreva os dados a partir de uma relação **linear**. 

![Disponivel em: https://www.researchgate.net/figure/Linear-Regression-model-sample-illustration_fig3_333457161](https://www.researchgate.net/profile/Hieu-Tran-17/publication/333457161/figure/fig3/AS:763959762247682@1559153609649/Linear-Regression-model-sample-illustration.ppm)

Para algoritmos de regressão, utilizaremos como alvo a coluna `'Y_num'` de `flights_vec`. 

In [ ]:
from pyspark.ml.regression import LinearRegression

print('Treinar e exibir resultados')

print('Avaliar')

print('Treinar e exibir resultados com regularização')


#### Random Forests

O algoritmo de Random Forests é um algoritmo de _ensemble_, ou seja, ele ajusta vários modelos que decidem a classe por meio de votação.

![Disponível em: https://en.wikipedia.org/wiki/Random_forest](https://upload.wikimedia.org/wikipedia/commons/7/76/Random_forest_diagram_complete.png)

Vamos executá-lo e comparar com Regressão Linear.

### Recomendação

ALgoritmos de recomendação capturam as interações dos usuário e produzem sugestões com base nelas. Em Spark apenas um único algoritmo está disponível dentro do módulo `pyspark.ml.recommendation` ([documentação](https://spark.apache.org/docs/latest/ml-collaborative-filtering.html)). Esse algoritmo é o chamado _Alternating Least Squares_ (`ALS`), uma forma de fatoração de matrizes.

![Disponível em: https://towardsdatascience.com/prototyping-a-recommender-system-step-by-step-part-2-alternating-least-square-als-matrix-4a76c58714a1](https://miro.medium.com/max/1838/1*xMxQL_V9CWeLggrk-Uyzmg.png)

Essa implementação suporta respostas explícitas e implícitas, e é robusta a _cold start_. Vamos fazer um exemplo usando o dataset [HetRec 2011 - MovieLens](https://grouplens.org/datasets/hetrec-2011/).

In [ ]:
#lendo dataset

#dividindo dataset

#rodando modelo


### Agrupamento

Algoritmos de agrupamento são utilizados quando queremos encontrar/aprender padrões em conjuntos de dados. Essa classe de algoritmos está no módulo `pyspark.ml.clustering` (veja a documentação [aqui](https://spark.apache.org/docs/latest/ml-pipeline.html)).

Como exemplo veremos o clássico `KMeans`, onde tenta-se particionar o conjunto de dados ao encontrar centróides que representam os dados.

![Disponivel em: https://medium.com/@luigi.fiori.lf0303/k-means-clustering-using-python-db57415d26e6](https://miro.medium.com/max/1200/1*TmvsQ4XaOxeb-TmKk1qgOw.png)


Podemos avaliar clusters com o objeto `ClusteringEvaluator`, que possui implementação da métrica de silhueta. Porém, agrupamento costumeiramente é melhor visualizado através de gráficos. Ao invés de avaliarmos a qualidade dos grupos por uma métrica, iremos fazer plotando os pontos e os centróides.

In [ ]:
#ler points.csv

#visualizar dataset

#rodar kmeans

#visualizar


## Pipelines

Até agora executamos cada passo separadamente, desde a preparação dos dados até a execução do algoritmo de ML. Spark oferece o recurso `Pipeline`, que permite unir todos os processos de transformação de características e o algoritmo em uma única execução. Você pode ver a documentação sobre pipelines [aqui](https://spark.apache.org/docs/latest/ml-pipeline.html). 

Pipelines são importantes pelo simples fato que auxiliam a evitar o problema de _vazamento de dados (data leakage)_, garantindo que todo o processo seja executado na partição de treinamento primeiro, e depois na partição de teste.

Vamos fazer o Pipeline para `flights_df`, encerrando com uma regressão linear.

In [ ]:
#divindo novamente a base

#construindo o pipeline

#checando os passos

#rodando o pipeline


Vamos fazer o pipeline para `sms_df`, terminando em uma regressão logística.

## Otimização e Validação Cruzada

Spark oferece também ferramentas de otimização e seleção de hiperparâmetros para ML, dentro do módulo `pyspark.ml.tuning` (veja a 
[documentação](https://spark.apache.org/docs/latest/ml-tuning.html)). Muitos algoritmos possuem vários hiperparâmetros que muitas das vezes só conseguem ser definidos empiricamente, rodando diversas configurações ou empregando técnicas de _busca em grade_. 

Para busca em grade, Spark oferece o objeto `ParamGridBuilder`. Esse recurso deve ser empregado em conjunto com um mecanismo de _Validação Cruzada_, executado com um `CrossValidator`. A Validação Cruzada é uma maneira muito eficiente de fazer otimização de parâmetros, pois é executada em diversas repartições dos dados de treinamento e oferece diversas visões de como um modelo se comporta. 

![Disponível em: https://drigols.medium.com/introdu%C3%A7%C3%A3o-a-valida%C3%A7%C3%A3o-cruzada-k-fold-2a6bced32a90](https://miro.medium.com/max/1202/0*O_491U1UfF1lIqz_.png)

Os módulos de otimização podem ser utilizados em conjunto com Pipeline. Vamos aproveitar os pipelines criados anteriormente e tentar otimizar os parâmetros dos algoritmos.

In [ ]:
# otimização para pipeline_flights

#criando um grid

#criando um avaliador

#rodando validação cruzada


Vamos ver os parâmetros escolhidos para o melhor modelo.

E finalmente, avaliar:

Para finalizar, vamos executar um cross validation no nosso `sms_df`. Aproveite o pipeline construído. Uma vantagem do `ParamGridBuilder` é que ele não se limita a ajustar parâmetros do algoritmo de ML. Tente ajustar o parâmetro de `HashingTF` também, além dos parâmetros selecionados para regressão logística.